<a href="https://colab.research.google.com/github/AbdurNawaz/Sentiment-RNN/blob/master/Sentiment_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from string import punctuation
import warnings
warnings.simplefilter('ignore')

In [0]:
with open('reviews.txt') as f:
    reviews = f.read()
with open('labels.txt') as f:
    labels = f.read()

In [0]:
text = ''.join([c for c in reviews if c not in punctuation])
text1 = text.split('\n')
text = ''.join(text1)
words = text.split()

In [0]:
vocab = sorted(set(words))
vocab_to_int = {v: i for i, v in enumerate(vocab, 1)}

reviews_int = []
for each in text1:
  reviews_int.append([vocab_to_int[word] for word in each.split()])

In [0]:
labels = labels.split('\n')
labels = np.array([1 if each is 'positive' else 0 for each in labels])

In [43]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_int])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [0]:
#Removing that one zero-length revies to avoid error later
indxs = [i for i, each in enumerate(reviews_int) if len(each) != 0]
reviews_int = [reviews_int[i] for i in indxs]
labels = [labels[i] for i in indxs]

In [0]:
#Clipping the reviews to 200 length to optimize the network
#and pad 0's to all the reviews with length less than 200
clip = 200
features = np.zeros((len(reviews_int), clip), dtype=int)
for i, each in enumerate(reviews_int):
  features[i, -len(each):] = np.array(each)[:clip]

In [0]:
split_frac = 0.8
split_indx = int(len(features)*split_frac)
x_train, x_valid = features[:split_indx], features[split_indx:]
y_train, y_valid = labels[:split_indx], labels[split_indx:]

test_indx = int(len(x_valid)*0.5)
x_test, x_valid = x_valid[:test_indx], x_valid[test_indx:]
y_test, y_valid = y_valid[:test_indx], y_valid[test_indx:]

In [0]:
lstm_size = 256
lstm_layers = 1
batch_size = 500
learning_rate = 0.01

In [0]:
n_words = len(vocab_to_int) + 1

graph = tf.Graph()

with graph.as_default():
  inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
  targets = tf.placeholder(tf.int32, [None, None], name='targets')
  keep_prob = tf.placeholder(tf.float32, name='keep_prob')
  

In [0]:
embedd_size = 200

with graph.as_default():
  embedding = tf.Variable(tf.random_uniform((n_words, embedd_size), -1, 1))
  embedd = tf.nn.embedding_lookup(embedding, inputs)

In [0]:
with graph.as_default():
  lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
  drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
  cell = tf.contrib.rnn.MultiRNNCell([lstm]*lstm_layers)
  initial_state = cell.zero_state(batch_size, tf.float32)


In [0]:
with graph.as_default():
  outputs, final_states = tf.nn.dynamic_rnn(cell, embedd, initial_state=initial_state)


In [0]:
with graph.as_default():
  predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
  cost = tf.losses.mean_squared_error(targets, predictions)
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [0]:
with graph.as_default():
  correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), targets)
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
def get_batches(x, y, batch_size=100):
  n_batches = len(x)//batch_size
  x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
  for i in range(0, len(x), batch_size):
    yield x[i:i+batch_size], y[i:i+batch_size]